In [1]:
import cv2
import mediapipe as mp
import pyautogui
import random
import util  # Assuming you have a 'util' module with necessary utility functions
from pynput.mouse import Button, Controller

# Initialize mouse controller
mouse_controller = Controller()

screen_w, screen_h = pyautogui.size()

mp_hand = mp.solutions.hands
hand_detector = mp_hand.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    max_num_hands=1
)

# Function to locate the index finger tip
def get_index_finger_tip(hand_data):
    if hand_data.multi_hand_landmarks:
        first_hand = hand_data.multi_hand_landmarks[0]  # Detecting only one hand
        index_tip = first_hand.landmark[mp_hand.HandLandmark.INDEX_FINGER_TIP]
        return index_tip
    return None, None

# Function to control mouse movement
def control_mouse_movement(finger_tip):
    if finger_tip:
        x_pos = int(finger_tip.x * screen_w)
        y_pos = int(finger_tip.y / 2 * screen_h)
        pyautogui.moveTo(x_pos, y_pos)

# Check if all fingers are extended for dragging
def check_all_fingers_extended(finger_landmarks):
    if finger_landmarks[8][1] < finger_landmarks[6][1] and \
       finger_landmarks[12][1] < finger_landmarks[10][1] and \
       finger_landmarks[16][1] < finger_landmarks[14][1] and \
       finger_landmarks[20][1] < finger_landmarks[18][1]:
        return True
    return False

# Function to detect a left-click gesture
def detect_left_click(finger_landmarks, thumb_distance):
    return (
        util.get_angle(finger_landmarks[5], finger_landmarks[6], finger_landmarks[8]) < 50 and
        util.get_angle(finger_landmarks[9], finger_landmarks[10], finger_landmarks[12]) > 90 and
        thumb_distance > 50
    )

# Function to detect a right-click gesture
def detect_right_click(finger_landmarks, thumb_distance):
    return (
        util.get_angle(finger_landmarks[9], finger_landmarks[10], finger_landmarks[12]) < 50 and
        util.get_angle(finger_landmarks[5], finger_landmarks[6], finger_landmarks[8]) > 90 and
        thumb_distance > 50
    )

# Function to detect a double-click gesture
def detect_double_click(finger_landmarks, thumb_distance):
    return (
        util.get_angle(finger_landmarks[5], finger_landmarks[6], finger_landmarks[8]) < 50 and
        util.get_angle(finger_landmarks[9], finger_landmarks[10], finger_landmarks[12]) < 50 and
        thumb_distance > 50
    )

# Function to detect a screenshot gesture
def detect_screenshot(finger_landmarks, thumb_distance):
    return (
        util.get_angle(finger_landmarks[5], finger_landmarks[6], finger_landmarks[8]) < 50 and
        util.get_angle(finger_landmarks[9], finger_landmarks[10], finger_landmarks[12]) < 50 and
        thumb_distance < 50
    )

# Function to process gestures and control mouse events
def process_gestures(frame, landmarks, hand_data):
    if len(landmarks) >= 21:
        index_tip = get_index_finger_tip(hand_data)
        thumb_index_dist = util.get_distance([landmarks[4], landmarks[5]])

        if check_all_fingers_extended(landmarks):
            # If all fingers are extended, start dragging
            pyautogui.mouseDown()
            control_mouse_movement(index_tip)
            cv2.putText(frame, "Dragging", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        else:
            pyautogui.mouseUp()  # Stop dragging when fingers close

        if thumb_index_dist < 50 and util.get_angle(landmarks[5], landmarks[6], landmarks[8]) > 90:
            control_mouse_movement(index_tip)
        elif detect_left_click(landmarks, thumb_index_dist):
            mouse_controller.press(Button.left)
            mouse_controller.release(Button.left)
            cv2.putText(frame, "Left Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        elif detect_right_click(landmarks, thumb_index_dist):
            mouse_controller.press(Button.right)
            mouse_controller.release(Button.right)
            cv2.putText(frame, "Right Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        elif detect_double_click(landmarks, thumb_index_dist):
            pyautogui.doubleClick()
            cv2.putText(frame, "Double Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        elif detect_screenshot(landmarks, thumb_index_dist):
            screenshot = pyautogui.screenshot()
            label = random.randint(1, 1000)
            screenshot.save(f'screenshot_{label}.png')
            cv2.putText(frame, "Screenshot Taken", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

# Main function for video capture and gesture detection
def run():
    drawing_utils = mp.solutions.drawing_utils
    video_capture = cv2.VideoCapture(0)

    try:
        while video_capture.isOpened():
            ret, frame = video_capture.read()
            if not ret:
                break
            frame = cv2.flip(frame, 1)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            hand_data = hand_detector.process(frame_rgb)

            landmarks = []
            if hand_data.multi_hand_landmarks:
                detected_hand = hand_data.multi_hand_landmarks[0]  # Only one hand is considered
                drawing_utils.draw_landmarks(frame, detected_hand, mp_hand.HAND_CONNECTIONS)
                for lm in detected_hand.landmark:
                    landmarks.append((lm.x, lm.y))

            process_gestures(frame, landmarks, hand_data)

            cv2.imshow('Hand Gesture Control', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        video_capture.release()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    run()

ModuleNotFoundError: No module named 'util'

In [20]:
import cv2
import mediapipe as mp
import pyautogui
import math
import random
from pynput.mouse import Button, Controller

# Initialize mouse controller
mouse_controller = Controller()

screen_w, screen_h = pyautogui.size()

mp_hand = mp.solutions.hands
hand_detector = mp_hand.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    max_num_hands=1
)

# Function to calculate the distance between two points
def get_distance(p1, p2):
    return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

# Function to calculate the angle between three points
def get_angle(p1, p2, p3):
    a = get_distance(p2, p3)
    b = get_distance(p1, p3)
    c = get_distance(p1, p2)
    if a * b == 0:  # Avoid division by zero
        return 180
    return math.degrees(math.acos((a**2 + b**2 - c**2) / (2 * a * b)))

# Function to locate the index finger tip
def get_index_finger_tip(hand_data):
    if hand_data.multi_hand_landmarks:
        first_hand = hand_data.multi_hand_landmarks[0]
        index_tip = first_hand.landmark[mp_hand.HandLandmark.INDEX_FINGER_TIP]
        return index_tip.x, index_tip.y
    return None

# Function to control mouse movement
def control_mouse_movement(finger_tip):
    if finger_tip:
        x_pos = int(finger_tip[0] * screen_w)  # Correct X scaling
        y_pos = int(finger_tip[1] * screen_h)  # Correct Y scaling
        pyautogui.moveTo(x_pos, y_pos)

# Check if all fingers are extended for dragging
def check_all_fingers_extended(finger_landmarks):
    return (
        finger_landmarks[8][1] < finger_landmarks[6][1] and
        finger_landmarks[12][1] < finger_landmarks[10][1] and
        finger_landmarks[16][1] < finger_landmarks[14][1] and
        finger_landmarks[20][1] < finger_landmarks[18][1]
    )

# Function to detect a left-click gesture
def detect_left_click(finger_landmarks, thumb_distance):
    return (
        get_angle(finger_landmarks[5], finger_landmarks[6], finger_landmarks[8]) < 50 and
        get_angle(finger_landmarks[9], finger_landmarks[10], finger_landmarks[12]) > 90 and
        thumb_distance > 50
    )

# Function to detect a right-click gesture
def detect_right_click(finger_landmarks, thumb_distance):
    return (
        get_angle(finger_landmarks[9], finger_landmarks[10], finger_landmarks[12]) < 50 and
        get_angle(finger_landmarks[5], finger_landmarks[6], finger_landmarks[8]) > 90 and
        thumb_distance > 50
    )

# Function to detect a double-click gesture
def detect_double_click(finger_landmarks, thumb_distance):
    return (
        get_angle(finger_landmarks[5], finger_landmarks[6], finger_landmarks[8]) < 50 and
        get_angle(finger_landmarks[9], finger_landmarks[10], finger_landmarks[12]) < 50 and
        thumb_distance > 50
    )

# Function to detect a screenshot gesture
def detect_screenshot(finger_landmarks, thumb_distance):
    return (
        get_angle(finger_landmarks[5], finger_landmarks[6], finger_landmarks[8]) < 50 and
        get_angle(finger_landmarks[9], finger_landmarks[10], finger_landmarks[12]) < 50 and
        thumb_distance < 50
    )

# Function to process gestures and control mouse events
def process_gestures(frame, landmarks, hand_data):
    if len(landmarks) >= 21:
        index_tip = get_index_finger_tip(hand_data)
        if index_tip is None:
            return

        thumb_index_dist = get_distance(landmarks[4], landmarks[5])

        if check_all_fingers_extended(landmarks):
            pyautogui.mouseDown()
            control_mouse_movement(index_tip)
            cv2.putText(frame, "Dragging", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        else:
            pyautogui.mouseUp()

        if thumb_index_dist < 50 and get_angle(landmarks[5], landmarks[6], landmarks[8]) > 90:
            control_mouse_movement(index_tip)
        elif detect_left_click(landmarks, thumb_index_dist):
            mouse_controller.press(Button.left)
            mouse_controller.release(Button.left)
            cv2.putText(frame, "Left Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        elif detect_right_click(landmarks, thumb_index_dist):
            mouse_controller.press(Button.right)
            mouse_controller.release(Button.right)
            cv2.putText(frame, "Right Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        elif detect_double_click(landmarks, thumb_index_dist):
            pyautogui.doubleClick()
            cv2.putText(frame, "Double Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        elif detect_screenshot(landmarks, thumb_index_dist):
            screenshot = pyautogui.screenshot()
            label = random.randint(1, 1000)
            screenshot.save(f'screenshot_{label}.png')
            cv2.putText(frame, "Screenshot Taken", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

# Main function for video capture and gesture detection
def run():
    drawing_utils = mp.solutions.drawing_utils
    video_capture = cv2.VideoCapture(0)

    try:
        while video_capture.isOpened():
            ret, frame = video_capture.read()
            if not ret:
                break
            frame = cv2.flip(frame, 1)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            hand_data = hand_detector.process(frame_rgb)

            landmarks = []
            if hand_data.multi_hand_landmarks:
                detected_hand = hand_data.multi_hand_landmarks[0]
                drawing_utils.draw_landmarks(frame, detected_hand, mp_hand.HAND_CONNECTIONS)
                for lm in detected_hand.landmark:
                    landmarks.append((lm.x, lm.y))

            process_gestures(frame, landmarks, hand_data)

            cv2.imshow('Hand Gesture Control', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        video_capture.release()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    run()


KeyboardInterrupt: 